## Import necessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.image import imread
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,classification_report,average_precision_score,confusion_matrix
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam
import cv2
from tensorflow.keras import layers
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping

In [ ]:
#pip install opencv-python

# Model 

In [ ]:
def build_model(pretrained):
    model = Sequential([
        pretrained, #DenseNet169
        layers.GlobalAveragePooling2D(),
        #layers.Dense(64,activation='relu'),
        #layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=0.0005),
        metrics=['accuracy']
    )
    
    return model

# Generator

In [ ]:
base_path = '/home/jovyan/work/real_vs_fake/real-vs-fake/'
image_gen = ImageDataGenerator(rescale=1./255.,
                               rotation_range=20,
                               #shear_range=0.2,
                               #zoom_range=0.2,
                               horizontal_flip=True)

train_flow = image_gen.flow_from_directory(
    base_path + 'train/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

In [ ]:
train_flow.class_indices

In [ ]:
image_gen1 = ImageDataGenerator(rescale=1./255.)

valid_flow = image_gen1.flow_from_directory(
    base_path + 'valid/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

# Train Model DenseNet Augmented Data

In [ ]:
densenet = DenseNet169(
    weights=None,
    include_top=False,
    input_shape=(224,224,3)
)
model = build_model(densenet)
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience = 1)

In [ ]:
train_steps = 100000//64
valid_steps = 20000//64

history = model.fit_generator(
    train_flow,
    epochs = 25,
    #callbacks=[early_stop],
    steps_per_epoch= train_steps,
    validation_data= valid_flow,
    validation_steps= valid_steps
)

In [ ]:

"""
Plot the training and validation loss
epochs - list of epoch numbers
loss - training loss for each epoch
val_loss - validation loss for each epoch
"""
def plot_loss(epochs, loss, val_loss):
    plt.plot(epochs, loss, 'bo', label='Training Loss')
    plt.plot(epochs, val_loss, 'orange', label = 'Validation Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()
"""
Plot the training and validation accuracy
epochs - list of epoch numbers
acc - training accuracy for each epoch
val_acc - validation accuracy for each epoch
"""
def plot_accuracy(epochs, acc, val_acc):
    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'orange', label = 'Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plot_loss(range(1, len(loss) + 1), loss, val_loss)
plot_accuracy(range(1, len(loss) + 1), acc, val_acc)

In [ ]:
test_flow = image_gen1.flow_from_directory(
    base_path + 'test/',
    target_size=(224, 224),
    batch_size=1,
    shuffle = False,
    class_mode='binary'
)
y_pred=model.predict(test_flow)
y_test = test_flow.classes

In [ ]:
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))
print("AP Score:", average_precision_score(y_test, y_pred))
print("Confusion Matrix : \n" ,confusion_matrix(y_test, y_pred>0.5))
print()
print(classification_report(y_test, y_pred > 0.5))

# Saving the model

In [ ]:
model.save("densenet169.h5")